In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import os
import matplotlib.image as img
import tqdm
from vae import Basic_VAE
from encoder import Encoder
from decoder import Decoder
from torch.utils.data import DataLoader, TensorDataset

#### Create Dataset


In [ ]:
cats = []
directory = "data/cats"
count = 0

for catpic in os.listdir(directory):
    if count < 1000:
        # read from image and convert to tensor
        im = torch.tensor(img.imread(os.path.join(directory, catpic))).float()
        # permute to (channels, height, width) for conv2d layer
        im = torch.permute(im, (2, 0, 1))
        # normalize to range between -1 and 1
        im = im / 128 - 1
        cats.append(im)
        count += 1
    else:
        break
cats = torch.stack(cats)
print(cats.shape)

#### Test/Training/Validation Split


In [ ]:
# Split the data into what we use for testing and not testing
training_test_split = 0.75
training_test_cutoff = int(cats.shape[0] * training_test_split + 1)
random_perm = torch.randperm(cats.shape[0])
not_test_tensor = cats[:training_test_cutoff]
testing_tensor = cats[training_test_cutoff:]
# Split the data into what we use for training and cross validation
training_cv_split = 0.8
training_cv_cutoff = int(not_test_tensor.shape[0] * training_cv_split)
training_tensor = not_test_tensor[:training_cv_cutoff]
cv_tensor = not_test_tensor[training_cv_cutoff:]
print(training_tensor.shape)
print(cv_tensor.shape)
print(testing_tensor.shape)

#### Choose Hyperparameters and Build Model


In [ ]:
from vae import Basic_VAE

hidden_dims = [16, 32, 64, 128]
latent_dim = 64
in_dim = 3
model = Basic_VAE(in_dim, hidden_dims, latent_dim)
# encoder = Encoder(in_dim, hidden_dims, latent_dim) for testing
# decoder = Decoder(latent_dim, hidden_dims) for testing

In [ ]:
# # Testing the model
model.forward(training_tensor)

# # Code to test out the encoder & decoder
# mu, log_var = encoder.forward(training_tensor)
# print("mu: ", mu.shape)
# print("log_var: ", log_var.shape)

# reconstructed_img = decoder.forward(mu, log_var)

encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])


[tensor([[[[ 0.6768,  0.7690, -0.0362,  ...,  0.1853,  0.1242,  0.0943],
           [ 0.0937,  0.6233,  0.8810,  ...,  0.2799,  0.2491, -0.4956],
           [ 0.3017,  0.3556,  0.2793,  ...,  0.7202,  0.5164,  0.1278],
           ...,
           [-0.1128,  0.0221, -0.5704,  ...,  0.8654,  0.4082, -0.1877],
           [ 0.0211,  0.2953,  0.4599,  ...,  0.5476, -0.0114,  0.2275],
           [ 0.1039,  0.4965,  0.3077,  ..., -0.0439,  0.1748,  0.2031]],
 
          [[-0.0504,  0.6194, -0.5115,  ...,  0.4151,  0.4376,  0.2416],
           [-0.7110, -0.9640, -0.8447,  ...,  0.8489, -0.8827,  0.0489],
           [-0.8370, -0.8408, -0.6446,  ...,  0.5381,  0.9613,  0.6388],
           ...,
           [-0.3912,  0.0827, -0.7018,  ...,  0.5258,  0.5986,  0.0882],
           [ 0.4367, -0.3559, -0.2975,  ..., -0.1871,  0.7205, -0.1976],
           [-0.0463, -0.1130,  0.7096,  ...,  0.2167,  0.2324,  0.4482]],
 
          [[ 0.4742, -0.2278, -0.2529,  ..., -0.2451,  0.0458, -0.3998],
           [-

#### Implement Loss Function


In [ ]:
def loss_function(reconstructed_img, input_img, mu, log_var, kld_weight=2):
    img_loss = F.mse_loss(reconstructed_img, input_img)
    # article on calculating kl divergence between 2 gaussians:
    # https://medium.com/@outerrencedl/variational-autoencoder-and-a-bit-kl-divergence-with-pytorch-ce04fd55d0d7
    # kld_loss = torch.mean(
    #     torch.sum(-log_var + (log_var.exp() ** 2 + mu**2) / 2 - 1 / 2)
        
    # )
    kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()))
    kld_loss *= kld_weight

    return img_loss + kld_loss

#### Implement Gradient Descent


In [ ]:
def gradient_descent(model, loss_func, x, y, xvalid, yvalid, lr=0.1, steps=5000, batch_size=32):
    # only really need x or y, they are the same thing
    optimizer = optim.AdamW(model.parameters(), lr)

    losses = []
    valid_losses = []

    train_dataset = TensorDataset(x, y)
    train_batches = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = TensorDataset(xvalid, yvalid)
    valid_batches = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    for _ in tqdm.trange(steps):
        model.train()
        total_loss = 0

        for input_batch, label_batch in train_batches:
            reconstructed_img, mu, log_var = model(input_batch)
            # y is the original image I think? yeah
            loss = loss_func(reconstructed_img, label_batch, mu, log_var)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            optimizer.zero_grad()
        mean_loss = total_loss/len(train_batches)
        losses.append(mean_loss)

        model.eval()
        total_valid_loss = 0
        for input_valid_batch, label_valid_batch in valid_batches:
            reconstructed_img, mu, log_var = model(input_valid_batch)
            valid_loss = loss_func(reconstructed_img, label_valid_batch, mu, log_var)
            total_valid_loss += valid_loss.detach()
        mean_valid_loss = total_valid_loss/len(valid_batches)
        valid_losses.append(mean_valid_loss)

    print(f"Final training loss: {losses[-1]}")

    return losses, valid_losses

In [ ]:
losses, valid_losses = gradient_descent(
    model, loss_function, training_tensor, training_tensor, cv_tensor, cv_tensor
)

  0%|          | 0/5000 [00:00<?, ?it/s]C:\Users\Joshua\AppData\Local\Temp\ipykernel_37472\241648366.py:4: UserWarning: Using a target size (torch.Size([600, 3, 64, 64])) that is different to the input size (torch.Size([600, 3, 8, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  img_loss = F.mse_loss(reconstructed_img, input_img)


encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])
reconstructed_img:  torch.Size([600, 3, 64, 64])
input_img:  torch.Size([600, 3, 64, 64])
encoded.shape: torch.Size([151, 128, 3, 3])
encoded.shape after flatten: torch.Size([151, 1152])


  0%|          | 1/5000 [00:06<9:10:45,  6.61s/it]

reconstructed_img:  torch.Size([151, 3, 64, 64])
input_img:  torch.Size([151, 3, 64, 64])
encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])
reconstructed_img:  torch.Size([600, 3, 64, 64])
input_img:  torch.Size([600, 3, 64, 64])
encoded.shape: torch.Size([151, 128, 3, 3])
encoded.shape after flatten: torch.Size([151, 1152])


  0%|          | 2/5000 [00:12<8:39:02,  6.23s/it]

...

encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])
reconstructed_img:  torch.Size([600, 3, 64, 64])
input_img:  torch.Size([600, 3, 64, 64])
encoded.shape: torch.Size([151, 128, 3, 3])
encoded.shape after flatten: torch.Size([151, 1152])


  0%|          | 4/5000 [00:18<5:42:26,  4.11s/it]

reconstructed_img:  torch.Size([151, 3, 64, 64])
input_img:  torch.Size([151, 3, 64, 64])
encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])


  0%|          | 4/5000 [00:20<6:57:08,  5.01s/it]


KeyboardInterrupt: 

In [ ]:
# after we've trained, we use the decoder to generate new images
# we assume a normal distribution over our latent space
# so we sample from that distribution and feed it into the decoder